# <font color= F30720> <b> <i> Modelando la serie de la temperatura </i> </b> </font>
La serie de la temperatura fue extraída de la base de datos del IDEAM, se tuvieron en cuenta los promedios diarios de las temperaturas en grados centigrados (°C) registradas en Bogotá en las diferentes estaciones meteorológicas que recolectaron información de esos días.
La serie de tiempo cuenta con un total de 1826 registros, de los cuales 13 (0.7%) fueron imputados puesto que no se presentaba la información necesaria. Esta imputación fue realizada a partir del método de vecinos más cercanos (KNN), donde se tuvierón en cuenta 5 vecinos.

Para el modelar la serie de la temperatura se usara inicialmente un modelo SARIMA, seguido de redes recurrentes simples, LSTM y finalmente GRU.
Para seleccionar el mejor modelo se tomará como criterio el error cuadrático medio.

## <font color= 199EDC> <b> Red Neuronal Recurrente Simple </b> </font>

#### <font color= blue>  Importación de Datos</font>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import impute
from sklearn import preprocessing
import sklearn.externals
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import KNNImputer
import sklearn.preprocessing

from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import time
import sklearn.externals
import joblib
import plotly.graph_objects as go
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.statespace.sarimax import SARIMAX

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Mineria /Temperatura1.csv", sep=';', header=0, decimal = ',')
Fecha = pd.date_range(start='2017-01-01', end='2021-12-31', freq='D')
df['Fecha'] = Fecha
df = df.set_index('Fecha')

print(df[pd.isnull(df.ValorObservado)])
print('En total hay' ,
      str(df['ValorObservado'].isnull().sum()) ,
      'valores sin información')
print('Correspondientes al {:.3f}% del total'
      .format(df['ValorObservado'].isnull().sum()*100/len(df)))

            ValorObservado
Fecha                     
2017-08-12             NaN
2017-12-24             NaN
2019-09-15             NaN
2019-09-16             NaN
2019-09-17             NaN
2020-11-12             NaN
2021-01-05             NaN
2021-01-06             NaN
2021-01-07             NaN
2021-01-08             NaN
2021-08-18             NaN
2021-08-20             NaN
2021-12-05             NaN
En total hay 13 valores sin información
Correspondientes al 0.712% del total


La serie presenta valores faltantes, por lo tantol se imputaran usando el método de vecinos más cercanos (KNN), como se muestra a continuación.

#### <font color= blue> Imputación a partir del vecino más cercano</font>

In [ ]:
#Imputación de Valores usando el vecino más cercano
imput = KNNImputer(n_neighbors=5, weights="uniform")

# Ajustamos el modelo e imputamos los missing values
imput.fit(df[['ValorObservado']])
df['ValorObservado'] = imput.transform(df[['ValorObservado']]).ravel()
print()
print("Valores pérdidos en ValorObservado: " , 
      str(df['ValorObservado'].isnull().sum()))


Valores pérdidos en ValorObservado:  0


In [ ]:
fig = px.line(df, x=df.index, y="ValorObservado")
fig.update_xaxes(title_text="Fecha")
fig.show()

#### <font color= blue> Separación de datos entrenamiento y validación  </font>
Para el respectivo análisis se tomarán el 80% de los datos para entrenamiento y validacion, el 20% restantes para prueba, dichos valores corresponden a 1460 y 365 respectivamente.

In [ ]:
# split data in 80%/20% train/test sets
test_set_size_percentage = 20

min_max_scaler = sklearn.preprocessing.MinMaxScaler()

# function for min-max normalization of stock
def normalize_data(df):
    df["ValorObservado"] = min_max_scaler.fit_transform(
        df.ValorObservado.values.reshape(-1, 1)
    )
    return df


# function to create train, test data given stock data and sequence length
def load_data(df, seq_len):
    data_raw = df.values  # convert to numpy array
    data = []

    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len):
        data.append(data_raw[index : index + seq_len])

    data = np.array(data)
    test_set_size = int(np.round(test_set_size_percentage / 100 * data.shape[0]))
    train_set_size = data.shape[0] - test_set_size

    X_train = data[:train_set_size, :-1, :]
    y_train = data[:train_set_size, -1, :]

    X_test = data[train_set_size:, :-1, :]
    y_test = data[train_set_size:, -1, :]

    return [X_train, y_train, X_test, y_test]


cols = list(df.columns.values)
print("df.columns.values = ", cols)

# normalize stock
df_norm = df.copy()
df_norm = normalize_data(df_norm)

# create train, test data
seq_len = 20  # choose sequence length
X_train, y_train, X_test, y_test = load_data(df_norm, seq_len)
print("X_train.shape = ", X_train.shape)
print("y_train.shape = ", y_train.shape)
print("X_test.shape = ", X_test.shape)
print("y_test.shape = ", y_test.shape)


df.columns.values =  ['ValorObservado']
X_train.shape =  (1445, 19, 1)
y_train.shape =  (1445, 1)
X_test.shape =  (361, 19, 1)
y_test.shape =  (361, 1)


In [ ]:
fig = px.line(df_norm, x=df.index, y="ValorObservado")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservadoNormalizado")
fig.show()

#### <font color= blue> Modelo (1 paso adelante) </font>

In [ ]:
# SRNN architecture
SRNN_model = Sequential()
# First GRU layer with Dropout regularisation
SRNN_model.add(L.SimpleRNN(units=50, input_shape=(seq_len - 1, 1), return_sequences=True))
SRNN_model.add(L.Dropout(0.2))
# Second GRU layer
SRNN_model.add(L.SimpleRNN(units=50, return_sequences=True))
SRNN_model.add(L.Dropout(0.2))
# Third GRU layer
SRNN_model.add(L.SimpleRNN(units=50, return_sequences=True))
SRNN_model.add(L.Dropout(0.2))
# Fourth GRU layer
SRNN_model.add(L.SimpleRNN(units=50))
SRNN_model.add(L.Dropout(0.5))
# The output layer
SRNN_model.add(L.Dense(units=50, kernel_initializer="uniform", activation="tanh"))
SRNN_model.add(L.Dense(units=1, kernel_initializer="uniform", activation="linear"))

earlystop = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=50)
callbacks_list = [earlystop]

# Compiling the RNN
SRNN_model.compile(optimizer="adam", loss="mean_squared_error")
# Fitting to the training set
start = time.time()
SRNN = SRNN_model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=35,
    validation_split=0.05,
    verbose=1,
    callbacks=callbacks_list,
)
print("compilation time : ", time.time() - start)

Epoch 1/100
40/40 [==============================] - 5s 40ms/step - loss: 0.0507 - val_loss: 0.0111
Epoch 2/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0188 - val_loss: 0.0144
Epoch 3/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0158 - val_loss: 0.0194
Epoch 4/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0137 - val_loss: 0.0150
Epoch 5/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0138 - val_loss: 0.0138
Epoch 6/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0130 - val_loss: 0.0134
Epoch 7/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0124 - val_loss: 0.0132
Epoch 8/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0110 - val_loss: 0.0116
Epoch 9/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0117 - val_loss: 0.0154
Epoch 10/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0110 - val_loss: 0.0124

In [ ]:
joblib.dump(SRNN_model,'/content/drive/Shareddrives/Mineria /SRNN')

['/content/drive/Shareddrives/Mineria /SRNN']

In [ ]:
SRNN_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_25 (SimpleRNN)   (None, 19, 50)            2600      
                                                                 
 dropout_24 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_26 (SimpleRNN)   (None, 19, 50)            5050      
                                                                 
 dropout_25 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_27 (SimpleRNN)   (None, 19, 50)            5050      
                                                                 
 dropout_26 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_28 (SimpleRNN)   (None, 50)               

In [ ]:
SRNN_losses = pd.DataFrame(SRNN.history)
fig = px.line(SRNN_losses, x=SRNN_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

#### <font color= blue> Predicciones (1 paso adelante)</font>

In [ ]:
SRNN_Predict = SRNN_model.predict(X_test)
SRNN_Predict = min_max_scaler.inverse_transform(SRNN_Predict)

12/12 [==============================] - 1s 8ms/step


In [ ]:
# plot y_train, y_test, and testPredict using plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=min_max_scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=min_max_scaler.inverse_transform(y_test).ravel(),
        mode="lines",
        name="Prueba",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=SRNN_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Intervalos de Confianza (1 paso adelante) </font>

In [ ]:
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - perc) + (1 - I) * perc
        # huber loss
        huber_loss = K.sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        return huber_loss + q_order_loss
    return _qloss

In [ ]:
# quantiles
perc_points = [0.025, 0.975]

# SRNN architecture
qSRNN_model = Sequential()
# First GRU layer with Dropout regularisation
qSRNN_model.add(L.SimpleRNN(units=50, input_shape=(seq_len - 1, 1), return_sequences=True))
qSRNN_model.add(L.Dropout(0.2))
# Second GRU layer
qSRNN_model.add(L.SimpleRNN(units=50, return_sequences=True))
qSRNN_model.add(L.Dropout(0.2))
# Third GRU layer
qSRNN_model.add(L.SimpleRNN(units=50, return_sequences=True))
qSRNN_model.add(L.Dropout(0.2))
# Fourth GRU layer
qSRNN_model.add(L.SimpleRNN(units=50))
qSRNN_model.add(L.Dropout(0.5))
# The output layer
qSRNN_model.add(L.Dense(units=50, kernel_initializer="uniform", activation="tanh"))
qSRNN_model.add(L.Dense(units=len(perc_points), kernel_initializer="uniform", activation="linear"))

earlystop = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=50)
callbacks_list = [earlystop]

# Compiling the RNN
qSRNN_model.compile(optimizer="adam", loss=QuantileLoss(perc_points))
# Fitting to the training set
start = time.time()
qSRNN = qSRNN_model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=35,
    validation_split=0.05,
    verbose=1,
    callbacks=callbacks_list,
)
print("compilation time : ", time.time() - start)

Epoch 1/100
40/40 [==============================] - 6s 40ms/step - loss: 0.1007 - val_loss: 0.0262
Epoch 2/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0222 - val_loss: 0.0171
Epoch 3/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0203 - val_loss: 0.0169
Epoch 4/100
40/40 [==============================] - 1s 25ms/step - loss: 0.0201 - val_loss: 0.0168
Epoch 5/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0181 - val_loss: 0.0173
Epoch 6/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0185 - val_loss: 0.0164
Epoch 7/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0184 - val_loss: 0.0174
Epoch 8/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0172 - val_loss: 0.0163
Epoch 9/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0173 - val_loss: 0.0168
Epoch 10/100
40/40 [==============================] - 1s 24ms/step - loss: 0.0171 - val_loss: 0.0188

In [ ]:
qSRNN_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_29 (SimpleRNN)   (None, 19, 50)            2600      
                                                                 
 dropout_28 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_30 (SimpleRNN)   (None, 19, 50)            5050      
                                                                 
 dropout_29 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_31 (SimpleRNN)   (None, 19, 50)            5050      
                                                                 
 dropout_30 (Dropout)        (None, 19, 50)            0         
                                                                 
 simple_rnn_32 (SimpleRNN)   (None, 50)               

In [ ]:
qSRNN_losses = pd.DataFrame(qSRNN.history)
fig = px.line(qSRNN_losses, x=qSRNN_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

In [ ]:
qSRNN_Predict = qSRNN_model.predict(X_test)
qSRNN_Predict = min_max_scaler.inverse_transform(qSRNN_Predict)

12/12 [==============================] - 1s 7ms/step


In [ ]:
# plot y_train, y_test, and testPredict using plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=min_max_scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=SRNN_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qSRNN_Predict[:,0] ,
        mode="lines",
        name="0.025",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qSRNN_Predict[:,1] ,
        mode="lines",
        name="0.975",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Error Cuadrático Medio (1 paso adelante) </font>

In [ ]:
SRNN_Score = metrics.mean_squared_error(y_test, SRNN_Predict) ** .5
print('Test Score: %.2f RMSE' % (SRNN_Score))

Test Score: 11.53 RMSE
